# Clase 1.3 - Herencia y Composición en Python

**Unidad:** 1 - POO Avanzada con Python  
**Conexión al Proyecto TaskFlow:** Diseñaremos la jerarquía de modelos del sistema TaskFlow usando herencia y composición.  

## 📋 Objetivos de Aprendizaje

Al finalizar esta clase, serás capaz de:
- [ ] Implementar herencia simple en Python
- [ ] Usar `super()` para llamar métodos de la clase padre
- [ ] Comprender el Method Resolution Order (MRO)
- [ ] Diferenciar entre herencia y composición
- [ ] Aplicar composición vs herencia en el diseño

## 💻 Conexión con TaskFlow

En el sistema **TaskFlow**, usaremos:
- **Herencia:** `BaseModel` → `Usuario`, `Proyecto`, `Tarea` (comparten id, creado_en)
- **Composición:** `Proyecto` contiene muchas `Tarea`s
- **Composición:** `Usuario` tiene muchas `Tarea`s asignadas

In [ ]:
# Importaciones
from dataclasses import dataclass, field
from datetime import datetime
from typing import List, Optional
from abc import ABC, abstractmethod

---

## 1. 🥳 Herencia Simple en Python

**📐 Definición:** La **herencia** es un mecanismo que permite a una clase (hija) heredar atributos y métodos de otra clase (padre).

**📋 Sintaxis:**
```python
class Padre:
    def metodo(self):
        print("Método del padre")

class Hija(Padre):
    def metodo_hija(self):
        print("Método de la hija")
```

**📈 Terminología:**
- **Clase Padre / Superclase / Base:** La clase que hereda sus características
- **Clase Hija / Subclase / Derivada:** La clase que hereda

In [ ]:
# Ejemplo 1: Herencia básica

@dataclass
class ModeloBase:
    """
    Clase base para todos los modelos de TaskFlow.
    Proporciona atributos comunes: id y timestamps.
    """
    id: Optional[int] = None
    creado_en: datetime = field(init=False, default_factory=datetime.now)
    actualizado_en: datetime = field(init=False, default_factory=datetime.now)
    
    def actualizar_timestamp(self) -> None:
        """Actualiza el timestamp de modificación."""
        self.actualizado_en = datetime.now()


@dataclass
class Usuario(ModeloBase):
    """Usuario del sistema TaskFlow."""
    username: str
    email: str
    nombre_completo: str = ""


@dataclass
class Proyecto(ModeloBase):
    """Proyecto en TaskFlow."""
    nombre: str
    descripcion: str
    usuario_id: Optional[int] = None


@dataclass
class Tarea(ModeloBase):
    """Tarea en TaskFlow."""
    titulo: str
    estado: str = "pendiente"
    proyecto_id: Optional[int] = None

# Crear instancias
u = Usuario(id=1, username="jdoe", email="john@example.com")
p = Proyecto(id=1, nombre="TaskFlow", descripcion="Sistema de gestión")
t = Tarea(id=1, titulo="Implementar auth")

# Todas tienen id, creado_en, actualizado_en (heredados)
print(f"Usuario: {u.username} - ID: {u.id} - Creado: {u.creado_en}")
print(f"Proyecto: {p.nombre} - ID: {p.id} - Creado: {p.creado_en}")
print(f"Tarea: {t.titulo} - ID: {t.id} - Creado: {t.creado_en}")

# Pueden usar métodos heredados
u.actualizar_timestamp()
print(f"\nTimestamp actualizado de Usuario: {u.actualizado_en}")

# Verificar herencia
print(f"\n¿Usuario es instancia de ModeloBase? {isinstance(u, ModeloBase)}")
print(f"¿Proyecto es instancia de ModeloBase? {isinstance(p, ModeloBase)}")

**💡 Ventaja:** No repetimos `id`, `creado_en`, `actualizado_en` en cada clase. Todo está centralizado en `ModeloBase`.

---

## 2. 🔹 Uso de `super()`

**📐 Definición:** `super()` es una función que permite acceder a métodos de la clase padre desde la clase hija. Es especialmente útil en `__init__` y al sobrescribir métodos.

**📋 ¿Cuándo usar `super()`?**
- En `__init__` para inicializar atributos del padre
- Al sobrescribir métodos para extender (no reemplazar) funcionalidad

**⚠️ Con dataclass:** `super().__init__()` no se llama explícitamente porque la dataclass lo hace automáticamente.

In [ ]:
# Ejemplo 2: super() con __post_init__

@dataclass
class ModeloBase:
    """Clase base con validación."""
    id: Optional[int] = None
    
    def __post_init__(self):
        """Validación base."""
        if self.id is not None and self.id < 0:
            raise ValueError("ID no puede ser negativo")


@dataclass
class Usuario(ModeloBase):
    """Usuario con validaciones adicionales."""
    username: str
    email: str
    
    def __post_init__(self):
        """
        Primero ejecuta validación del padre, luego las propias.
        """
        # Llamar a __post_init__ del padre
        super().__post_init__()
        
        # Validaciones propias
        if len(self.username) < 3:
            raise ValueError("Username debe tener al menos 3 caracteres")
        if "@" not in self.email:
            raise ValueError("Email debe contener @")

# Crear usuario válido
u1 = Usuario(id=1, username="jdoe", email="john@example.com")
print(f"Usuario válido creado: {u1.username}")

# Intentar con ID negativo (error del padre)
try:
    u2 = Usuario(id=-1, username="asmith", email="alice@example.com")
except ValueError as e:
    print(f"\nError de validación (padre): {e}")

# Intentar con username corto (error del hijo)
try:
    u3 = Usuario(id=2, username="ab", email="bob@example.com")
except ValueError as e:
    print(f"Error de validación (hijo): {e}")

**💡 Flujo:** `Usuario.__post_init__()` → `super().__post_init__()` → `ModeloBase.__post_init__()` → vuelve a `Usuario.__post_init__()` → validaciones adicionales

---

## 3. 🔍 Method Resolution Order (MRO)

**📐 Definición:** El **MRO** es el orden en que Python busca métodos y atributos en una jerarquía de clases.

**📋 Para ver el MRO:**
```python
MiClase.mro()  # Retorna lista
MiClase.__mro__  # Tupla
```

In [ ]:
# Ejemplo 3: MRO en acción

class A:
    def metodo(self):
        print("Método de A")
        return "A"

class B(A):
    def metodo(self):
        print("Método de B")
        return "B"

class C(B):
    pass

# Ver el MRO de C
print("MRO de C:")
for i, cls in enumerate(C.mro(), 1):
    print(f"  {i}. {cls.__name__}")

# Crear instancia y llamar metodo
c = C()
resultado = c.metodo()
print(f"\nResultado: {resultado}")
print("\nExplicación: Python busca en C \u2192 B (encuentra aqu) y se detiene.")

In [ ]:
# Ejemplo 4: MRO con herencia múltiple (avanzado)

class A:
    def metodo(self):
        return "A"

class B(A):
    def metodo(self):
        return "B"

class C(A):
    def metodo(self):
        return "C"

class D(B, C):  # Hereda de B y C (ambos heredan de A)
    pass

print("MRO de D:")
for i, cls in enumerate(D.mro(), 1):
    print(f"  {i}. {cls.__name__}")

d = D()
print(f"\nResultado: {d.metodo()}")
print("\nExplicación: D \u2192 B (encuentra) \u2192 se detiene.")

**💡 Nota:** Python usa **C3 linearization** para calcular el MRO, lo que garantiza un orden consistente y predecible.

---

## 4. 🔷 Herencia vs Composición

**📐 Diferencia clave:**

| Aspecto | Herencia ("es-un") | Composición ("tiene-un") |
|---------|---------------------|------------------------|
| **Relación** | Es un tipo de | Contiene |
| **Ejemplo** | `Usuario` **es un** `ModeloBase` | `Proyecto` **tiene** `Tarea`s |
| **Símbolo UML** | → (flecha vacía) | ⚪ (diamante sólido) |
| **Acoplamiento** | Alto (fuerte) | Bajo (débil) |
| **Flexibilidad** | Menos flexible | Más flexible |

**📋 Regla general:**
- Usa **herencia** cuando hay una relación "es-un" clara
- Usa **composición** cuando hay una relación "tiene-un"

**⚠️ Anti-patrón:** "Forzar herencia" cuando no hay relación "es-un" real.

In [ ]:
# Ejemplo 5: Composición en TaskFlow

@dataclass
class Tarea:
    """Tarea individual."""
    id: Optional[int] = None
    titulo: str = ""
    estado: str = "pendiente"


@dataclass
class Proyecto:
    """Proyecto que CONTIENE tareas (composición)."""
    id: Optional[int] = None
    nombre: str = ""
    descripcion: str = ""
    tareas: List[Tarea] = field(init=False, default_factory=list)
    
    def agregar_tarea(self, tarea: Tarea) -> None:
        """Agrega una tarea al proyecto."""
        self.tareas.append(tarea)
    
    def eliminar_tarea(self, tarea_id: int) -> bool:
        """
        Elimina una tarea por ID.
        
        Returns:
            True si se eliminó, False si no se encontró
        """
        for i, tarea in enumerate(self.tareas):
            if tarea.id == tarea_id:
                self.tareas.pop(i)
                return True
        return False
    
    def tareas_pendientes(self) -> List[Tarea]:
        """Retorna las tareas pendientes."""
        return [t for t in self.tareas if t.estado == "pendiente"]
    
    def tareas_completadas(self) -> List[Tarea]:
        """Retorna las tareas completadas."""
        return [t for t in self.tareas if t.estado == "completada"]
    
    def progreso(self) -> float:
        """
        Calcula el porcentaje de progreso.
        
        Returns:
            Porcentaje completado (0-100)
        """
        if not self.tareas:
            return 0.0
        completadas = len(self.tareas_completadas())
        return (completadas / len(self.tareas)) * 100

# Crear proyecto y tareas
proyecto = Proyecto(id=1, nombre="TaskFlow", descripcion="Sistema de gestión")

# Agregar tareas (composición)
proyecto.agregar_tarea(Tarea(id=1, titulo="Diseñar BD", estado="completada"))
proyecto.agregar_tarea(Tarea(id=2, titulo="Crear modelos", estado="completada"))
proyecto.agregar_tarea(Tarea(id=3, titulo="Implementar API", estado="en_progreso"))
proyecto.agregar_tarea(Tarea(id=4, titulo="Crear UI", estado="pendiente"))

# Ver información
print(f"Proyecto: {proyecto.nombre}")
print(f"Total tareas: {len(proyecto.tareas)}")
print(f"Tareas pendientes: {len(proyecto.tareas_pendientes())}")
print(f"Tareas completadas: {len(proyecto.tareas_completadas())}")
print(f"Progreso: {proyecto.progreso():.1f}%")

**💡 Nota:** `Proyecto` no es un tipo de `Tarea`, sino que **contiene** tareas. Esta es la clave de la composición.

---

## 5. 🔷 Ejemplo Completo: Herencia + Composición

Combinemos ambos conceptos en el sistema TaskFlow completo.

In [ ]:
# Ejemplo 6: TaskFlow completo con herencia y composición

# === HERENCIA: Modelo base ===
@dataclass
class ModeloBase:
    """Clase base para todos los modelos."""
    id: Optional[int] = None
    creado_en: datetime = field(init=False, default_factory=datetime.now)
    
    def __str__(self) -> str:
        """Representación string base."""
        return f"{self.__class__.__name__}(id={self.id})"


# === HERENCIA: Usuario ===
@dataclass
class Usuario(ModeloBase):
    """Usuario del sistema."""
    username: str
    email: str
    nombre_completo: str = ""


# === HERENCIA: Proyecto ===
@dataclass
class Proyecto(ModeloBase):
    """Proyecto que contiene tareas (composición)."""
    nombre: str
    descripcion: str = ""
    propietario: Optional[Usuario] = None  # Composición
    tareas: List['Tarea'] = field(init=False, default_factory=list)
    
    def agregar_tarea(self, tarea: 'Tarea') -> None:
        self.tareas.append(tarea)
        tarea.proyecto = self
    
    def progreso(self) -> float:
        if not self.tareas:
            return 0.0
        completadas = sum(1 for t in self.tareas if t.estado == "completada")
        return (completadas / len(self.tareas)) * 100


# === HERENCIA: Tarea ===
@dataclass
class Tarea(ModeloBase):
    """Tarea que pertenece a un proyecto (composición)."""
    titulo: str
    descripcion: str = ""
    estado: str = "pendiente"
    asignado_a: Optional[Usuario] = None  # Composición
    proyecto: Optional[Proyecto] = field(init=False, default=None)

# Crear usuarios
juan = Usuario(id=1, username="jdoe", email="john@example.com", nombre_completo="John Doe")
maria = Usuario(id=2, username="asmith", email="alice@example.com", nombre_completo="Alice Smith")

# Crear proyecto (composición: tiene propietario)
proyecto = Proyecto(id=1, nombre="TaskFlow", descripcion="Sistema de gestión", propietario=juan)

# Crear tareas (composición: asignadas a usuarios, pertenecen a proyecto)
t1 = Tarea(id=1, titulo="Diseñar BD", estado="completada", asignado_a=juan)
t2 = Tarea(id=2, titulo="Crear modelos", estado="completada", asignado_a=maria)
t3 = Tarea(id=3, titulo="Implementar API", estado="en_progreso", asignado_a=maria)
t4 = Tarea(id=4, titulo="Crear UI", estado="pendiente", asignado_a=juan)

# Agregar tareas al proyecto
for tarea in [t1, t2, t3, t4]:
    proyecto.agregar_tarea(tarea)

# Mostrar información
print(f"\n{'='*50}")
print(f"PROYECTO: {proyecto.nombre}")
print(f"Propietario: {proyecto.propietario.nombre_completo}")
print(f"Descripción: {proyecto.descripcion}")
print(f"Progreso: {proyecto.progreso():.1f}%")

print(f"\nTAREAS ({len(proyecto.tareas)}):")
for tarea in proyecto.tareas:
    asignado = tarea.asignado_a.nombre_completo if tarea.asignado_a else "Sin asignar"
    print(f"  [{tarea.estado.upper()}] {tarea.titulo} - Asignado a: {asignado}")

print(f"\n{'='*50}")
print("RELACIONES:")
print(f"  - Proyecto ES UN ModeloBase: {isinstance(proyecto, ModeloBase)}")
print(f"  - Tarea ES UN ModeloBase: {isinstance(t1, ModeloBase)}")
print(f"  - Proyecto TIENE tareas (composición): {len(proyecto.tareas)} tareas")
print(f"  - Proyecto TIENE propietario (composición): {proyecto.propietario.username}")

**💡 Diagrama de clases simplificado:**
```
          ModeloBase
              / | \
             /  |  \
        Usuario  Proyecto  Tarea
                  |       ^
                  |       |
              tiene --- pertenece
```

---

## 6. 📋 Cuándo Usar Herencia vs Composición

### ✅ Usa HERENCIA cuando:

1. **Existe relación "es-un" clara:**
   ```python
   class Gerente(Usuario):  # Gerente ES UN Usuario
       pass
   ```

2. **Quieres reutilizar código entre clases relacionadas:**
   ```python
   class ModeloBase:
       id: int
   ```

3. **Quieres polimorfismo (mismo método, diferente comportamiento):**
   ```python
   class Figura:
       def area(self): pass
   ```

### ✅ Usa COMPOSICIÓN cuando:

1. **Existe relación "tiene-un" clara:**
   ```python
   class Proyecto:
       tareas: List[Tarea]  # Proyecto TIENE tareas
   ```

2. **Quieres cambiar componentes en tiempo de ejecución:**
   ```python
   class Proyecto:
       def cambiar_propietario(self, nuevo_usuario: Usuario):
           self.propietario = nuevo_usuario
   ```

3. **Quieres menor acoplamiento:**
   ```python
   # Composición: más flexible
   class Motor:
       pass
   class Auto:
       motor: Motor  # Se puede cambiar
   ```

---

## 📝 Ejercicio Práctico: Jerarquía de Modelos

Completa la jerarquía de modelos para TaskFlow:

1. `ModeloBase`: Clase base con `id`, `creado_en`, `__str__`
2. `Usuario(ModeloBase)`: `username`, `email`, método `es_admin()`
3. `Administrador(Usuario)`: Sobrescribe `es_admin()` para retornar `True`
4. `Proyecto(ModeloBase)`: `nombre`, `propietario` (Usuario), método `puede_editar(usuario)`
5. `Tarea(ModeloBase)`: `titulo`, `proyecto` (Proyecto), `asignado_a` (Usuario)

In [ ]:
# Tu solución aquí
from dataclasses import dataclass, field
from datetime import datetime
from typing import Optional

# TODO: Completar la jerarquía

@dataclass
class ModeloBase:
    """Clase base para todos los modelos."""
    id: Optional[int] = None
    creado_en: datetime = field(init=False, default_factory=datetime.now)
    
    # TODO: Agregar método __str__
    pass

@dataclass
class Usuario(ModeloBase):
    """Usuario del sistema."""
    username: str
    email: str
    
    # TODO: Completar método es_admin()
    def es_admin(self) -> bool:
        """Retorna True si el usuario es administrador."""
        pass

@dataclass
class Administrador(Usuario):
    """Administrador del sistema."""
    
    # TODO: Sobrescribir es_admin() para retornar True
    pass

@dataclass
class Proyecto(ModeloBase):
    """Proyecto con propietario."""
    nombre: str
    propietario: Optional[Usuario] = None
    
    # TODO: Completar método puede_editar
    def puede_editar(self, usuario: Usuario) -> bool:
        """
        Verifica si un usuario puede editar el proyecto.
        Solo el propietario o un administrador pueden editar.
        """
        pass

### ✅ Validación Automática

In [ ]:
# Tests automáticos

def test_jerarquia():
    """Valida la jerarquía de modelos."""
    
    # Test 1: Crear usuarios
    usuario = Usuario(id=1, username="jdoe", email="john@example.com")
    admin = Administrador(id=2, username="admin", email="admin@example.com")
    
    # Verificar herencia
    assert isinstance(usuario, ModeloBase), "Usuario debe heredar de ModeloBase"
    assert isinstance(admin, Usuario), "Administrador debe heredar de Usuario"
    assert isinstance(admin, ModeloBase), "Administrador debe heredar de ModeloBase (indirectamente)"
    print("✅ Test 1: Herencia correcta")
    
    # Test 2: Método es_admin
    assert not usuario.es_admin(), "Usuario normal no es admin"
    assert admin.es_admin(), "Administrador es admin"
    print("✅ Test 2: Método es_admin funciona")
    
    # Test 3: Proyecto y permisos
    proyecto = Proyecto(id=1, nombre="TaskFlow", propietario=usuario)
    assert proyecto.puede_editar(usuario), "Propietario puede editar"
    assert proyecto.puede_editar(admin), "Admin puede editar cualquier proyecto"
    
    otro_usuario = Usuario(id=3, username="other", email="other@example.com")
    assert not proyecto.puede_editar(otro_usuario), "Otro usuario no puede editar"
    print("✅ Test 3: Permisos de edición correctos")
    
    # Test 4: Método __str__
    str_usuario = str(usuario)
    assert "Usuario" in str_usuario and "1" in str_usuario
    print("✅ Test 4: Método __str__ funciona")
    
    print("\n🚀 ¡Todos los tests pasaron!")

# Ejecutar tests
try:
    test_jerarquia()
except (AssertionError, AttributeError) as e:
    print(f"\n❌ {e}")
    print("\nRevisa tu implementación.")

### 📋 Solución del Ejercicio

In [ ]:
# Solución completa

@dataclass
class ModeloBase:
    """Clase base para todos los modelos."""
    id: Optional[int] = None
    creado_en: datetime = field(init=False, default_factory=datetime.now)
    
    def __str__(self) -> str:
        """Representación string."""
        return f"{self.__class__.__name__}(id={self.id})"


@dataclass
class Usuario(ModeloBase):
    """Usuario del sistema."""
    username: str
    email: str
    
    def es_admin(self) -> bool:
        """Retorna True si el usuario es administrador."""
        return False  # Por defecto, no es admin


@dataclass
class Administrador(Usuario):
    """Administrador del sistema."""
    
    def es_admin(self) -> bool:
        """Sobrescribe para retornar siempre True."""
        return True


@dataclass
class Proyecto(ModeloBase):
    """Proyecto con propietario."""
    nombre: str
    propietario: Optional[Usuario] = None
    
    def puede_editar(self, usuario: Usuario) -> bool:
        """
        Verifica si un usuario puede editar el proyecto.
        Solo el propietario o un administrador pueden editar.
        """
        # Es admin o es el propietario
        return usuario.es_admin() or (self.propietario and usuario.id == self.propietario.id)

# Ejecutar tests de nuevo
test_jerarquia()

---

## 📈 Diagrama: Herencia vs Composición

```mermaid
graph TB
    subgraph["HERENCIA: 'es-un'"]
        A[ModeloBase] --> B[Usuario]
        A --> C[Proyecto]
        A --> D[Tarea]
        B --> E[Administrador]
    end
    
    subgraph["COMPOSICIÓN: 'tiene-un'"]
        C -.tiene.-> F[Lista de Tareas]
        C -.propiedad de.-> B
        D -.asignada a.-> B
        D -.pertenece a.-> C
    end
    
    style A fill:#e1f5ff
    style E fill:#c8e6c9
    style F fill:#fff9c4
```

---

## 📝 Resumen de la Clase

### 📋 Conceptos Clave

| Concepto | Propósito | Ejemplo TaskFlow |
|----------|-----------|------------------|
| **Herencia** | Reutilizar código, relación "es-un" | `Usuario` es un `ModeloBase` |
| **super()** | Llamar métodos de la clase padre | `super().__post_init__()` |
| **MRO** | Orden de búsqueda de métodos | `Clase.mro()` |
| **Composición** | Relación "tiene-un", más flexible | `Proyecto` tiene `Tarea`s |

### 📋 Reglas de Diseño

1. **✅ Usa herencia** cuando existe una clara relación "es-un"
2. **✅ Usa composición** cuando existe una clara relación "tiene-un"
3. **⚠️ Evita herencia profunda** (más de 3 niveles)
4. **✅ Prefiere composición** sobre herencia cuando sea posible
5. **✅ Usa `super()`** para extender, no reemplazar funcionalidad

### 🤝 Conexión con TaskFlow

Hemos diseñado:
- `ModeloBase` con atributos comunes (id, timestamps)
- `Usuario`, `Proyecto`, `Tarea` heredando de `ModeloBase`
- `Administrador` heredando de `Usuario`
- `Proyecto` compuesto de `Tarea`s
- `Tarea` compuesta de referencias a `Usuario` y `Proyecto`

**📈 Próxima clase:** Polimorfismo y Métodos Mágicos

Aprenderemos a:
- Implementar `__str__` y `__repr__`
- Sobrecargar operadores (`__add__`, `__eq__`, etc.)
- Aplicar duck typing
- Comparar objetos por prioridad